In [1]:
#Add Dependencies
import pandas as pd
import numpy as np
import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### Connection to the Postgres database and Retrive data from the tables

In [3]:
#Read Data from the PostgressSql
conn = pg.connect(user = "postgres",
                                  password = "akshaj10",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "movie_data")
sql1 = "Select * From movie_metadata;"
sql2 = "Select * from award;"
movie_df = pd.read_sql_query(sql1, conn)
award_df = pd.read_sql_query(sql2, conn)
conn = None

In [4]:
#Load Data into Pandas Data frame and Display the dataframe.
movie_df.shape

(45006, 13)

In [5]:
#Load Data into Pandas Data frame and Display the dataframe.
award_df.shape

(10395, 4)

In [6]:
#Merge the two dataframe
combined_df = pd.merge(movie_df, award_df,how='left', left_on='title', right_on='film' )
combined_df.shape

(51598, 17)

In [7]:
# Filter all rows for US as the country
movie_us_df = combined_df[combined_df['country'] == 'United States of America'] 
movie_us_df.shape

(22651, 17)

In [8]:
# Find null values
for column in movie_us_df.columns:
    print(f"Column{column} has {movie_us_df[column].isnull().sum()} null values")

Columnimdb_id has 3 null values
Columnbudget has 0 null values
Columnoriginal_title has 0 null values
Columnpopularity has 0 null values
Columnrevenue has 0 null values
Columnruntime has 5 null values
Columntitle has 0 null values
Columnvote_average has 0 null values
Columnvote_count has 0 null values
Columnproduction has 0 null values
Columncountry has 0 null values
Columnnew_genre has 0 null values
ColumnRelease_year has 2 null values
Columnindex has 15911 null values
Columncategory has 15911 null values
Columnfilm has 15911 null values
Columnwinner has 15911 null values


In [9]:
#Set the index to IMDB_ID
movie_us_df = movie_us_df.set_index("imdb_id")
movie_us_df.head()

,budget,original_title,popularity,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,Release_year,index,category,film,winner
imdb_id,,,,,,,,,,,,,,,,
tt0114709,30000000,Toy Story,30000000.0,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,7508.0,MUSIC (Original Musical or Comedy Score),Toy Story,False
tt0114709,30000000,Toy Story,30000000.0,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,7514.0,MUSIC (Original Song),Toy Story,False
tt0114709,30000000,Toy Story,30000000.0,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,7548.0,WRITING (Screenplay Written Directly for the S...,Toy Story,False
tt0113497,65000000,Jumanji,65000000.0,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,1995,NaN,NaN,NaN,NaN
tt0113228,0,Grumpier Old Men,0.0,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,1995,NaN,NaN,NaN,NaN


In [10]:
columns =['budget','original_title','popularity','revenue','runtime','vote_average','vote_count','production','country','new_genre','Release_year', 'winner']

target = ['winner']

In [11]:
movie_us_df = movie_us_df.loc[:, columns].copy()
movie_us_df.shape

(22651, 12)

In [12]:
#Convert winner column into numbers 
movie_us_df["winner"] = movie_us_df["winner"].replace({True: 1, False: 0})
movie_us_df.shape

(22651, 12)

In [13]:
#Fill null values in winner column to zero 
movie_us_df["winner"] = movie_us_df["winner"].fillna(value=0)
movie_us_df.shape

(22651, 12)

In [14]:
movie_us_df['Release_year']=movie_us_df.popularity.astype('int64')

In [15]:
# Create a title Dataframe.
title_df=pd.DataFrame(data=movie_us_df['original_title'],index=movie_us_df.index,copy=True)
title_df.head()

,original_title
imdb_id,
tt0114709,Toy Story
tt0114709,Toy Story
tt0114709,Toy Story
tt0113497,Jumanji
tt0113228,Grumpier Old Men


In [16]:
# Remove the movie title from the dataframe
movie_us_df = movie_us_df.drop('original_title',1)
movie_us_df.head()

,budget,popularity,revenue,runtime,vote_average,vote_count,production,country,new_genre,Release_year,winner
imdb_id,,,,,,,,,,,
tt0114709,30000000,30000000.0,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,30000000,0.0
tt0114709,30000000,30000000.0,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,30000000,0.0
tt0114709,30000000,30000000.0,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,30000000,0.0
tt0113497,65000000,65000000.0,262797249.0,104.0,6.9,2413.0,TriStar Pictures,United States of America,Adventure,65000000,0.0
tt0113228,0,0.0,0.0,101.0,6.5,92.0,Warner Bros.,United States of America,Romance,0,0.0


In [17]:
# Drop null rows
movie_us_df = movie_us_df.dropna(axis = 0,how= 'any')
movie_us_df.shape

(22646, 11)

In [18]:
# Generate our categorical variable list for movie dataset
movie_cat = movie_us_df.dtypes[movie_us_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
movie_us_df[movie_cat].nunique()

production    4610
country          1
new_genre       20
dtype: int64

### One Hot encoding production -Using top 10 values

In [19]:
# Check the production unique value counts to see if binning is required
production_counts= movie_us_df.production.value_counts()
production_counts

Other                                     2314
Paramount Pictures                        1374
Metro-Goldwyn-Mayer (MGM)                 1167
Twentieth Century Fox Film Corporation    1125
Warner Bros.                               972
                                          ... 
Hughes Capital Entertainment                 1
Steeltown Entertainment                      1
Archer Gray                                  1
Ketchum Labs                                 1
Cinema Venture                               1
Name: production, Length: 4610, dtype: int64

In [20]:
# Determine which values to replace for bucketing
replace_production = list(production_counts[production_counts < 42].index)

# Replace in DataFrame
for production in replace_production:
    movie_us_df.production = movie_us_df.production.replace(production,"Other")


# Check to make sure binning was successful
movie_us_df.production.value_counts()

Other                                     12383
Paramount Pictures                         1374
Metro-Goldwyn-Mayer (MGM)                  1167
Twentieth Century Fox Film Corporation     1125
Warner Bros.                                972
Universal Pictures                          820
Columbia Pictures                           530
Columbia Pictures Corporation               480
RKO Radio Pictures                          423
United Artists                              393
Walt Disney Pictures                        287
New Line Cinema                             226
Miramax Films                               210
TriStar Pictures                            177
Orion Pictures                              121
Touchstone Pictures                         106
Walt Disney Productions                     106
Samuel Goldwyn Company, The                 105
DreamWorks SKG                               93
Universal International Pictures (UI)        89
Fox Searchlight Pictures                

In [21]:
# let's examine how many columns we will obtain after one hot encoding these variables
encode_df=pd.get_dummies(movie_us_df[movie_cat], drop_first=True)
encode_df.shape

(22646, 64)

In [22]:
# Merge one-hot encoded features and drop the originals
oscar_us_df = movie_us_df.merge(encode_df,left_index=True, right_index=True)
oscar_us_df = oscar_us_df.drop(movie_cat,1)
oscar_us_df.head(2)

,budget,popularity,revenue,runtime,vote_average,vote_count,Release_year,winner,production_Amblin Entertainment,production_American International Pictures (AIP),...,new_genre_History,new_genre_Horror,new_genre_Music,new_genre_Mystery,new_genre_Romance,new_genre_Science Fiction,new_genre_TV Movie,new_genre_Thriller,new_genre_War,new_genre_Western
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0000001,0,0.0,0.0,1.0,4.9,18.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000005,0,0.0,0.0,1.0,5.8,19.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Decision Tree

In [23]:
# Define the features set.
X = oscar_us_df.copy()
X = X.drop('winner', axis=1)
X.shape

(52646, 71)

In [24]:
# Define the target set(output label)
y = oscar_us_df['winner']
y.head(2)

imdb_id
tt0000001    0.0
tt0000005    0.0
Name: winner, dtype: float64

In [25]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(39484, 71)
(13162, 71)
(39484,)
(13162,)


In [27]:
# Creating a StandardScaler instance.
scaler = StandardScaler()

In [28]:
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [29]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [31]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [32]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm_df

,Predicted 0,Predicted 1
Actual 0,10161,576
Actual 1,1543,882


In [33]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8390062300562224

In [34]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,10161,576
Actual 1,1543,882


Accuracy Score : 0.8390062300562224
Classification Report
              precision    recall  f1-score   support

         0.0       0.87      0.95      0.91     10737
         1.0       0.60      0.36      0.45      2425

    accuracy                           0.84     13162
   macro avg       0.74      0.66      0.68     13162
weighted avg       0.82      0.84      0.82     13162

